<div style="color:white;display:fill;
            background-color:#3f4d6f;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b> | Importing packages</b></p>
</div>

In [1]:
import numpy as np 
import pandas as pd 
import sklearn 
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
import random
from sklearn.linear_model import LogisticRegression



from sklearn.linear_model import SGDClassifier, LogisticRegression



import os
os.chdir('C:/Users/User/Desktop/data')
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)



<div style="color:white;display:fill;
            background-color:#3f4d6f;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b> | define functions</b></p>
</div>

In [2]:
import re
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import stem
from sklearn import metrics

# Define preprocess function for Arabic text
def preprocess_arabic_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters and numbers
    text = re.sub(r'[^ء-ي\s]', ' ', text)  # Adjust the range based on your specific needs
    # Remove emojis and non-word characters
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove Twitter handles
    text = re.sub(r'@[^\s]+', '', text)
    # Remove hashtags
    text = re.sub(r'\B#\S+', '', text)
    # Remove single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', '', text)
    # Substituting multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    # Tokenization
    words = text.split()
    # Remove stopwords
    stop_words = set(stopwords.words('arabic'))
    words = [word for word in words if word not in stop_words]
    # Join the words back into a string
    processed_text = ' '.join(words)
    return processed_text

# Define function to read TSV file
def read_tsv(data_file):
    text_data = list()
    labels = list()
    infile = open(data_file, encoding='utf-8')
    for line in infile:
        if not line.strip():
            continue
        label, text = line.split('\t')
        text_data.append(text)
        labels.append(label)
    return text_data, labels

# Load data from files
def load(pos_train_file, neg_train_file, pos_test_file, neg_test_file):
    pos_train_data, pos_train_labels = read_tsv(pos_train_file)
    neg_train_data, neg_train_labels = read_tsv(neg_train_file)

    pos_test_data, pos_test_labels = read_tsv(pos_test_file)
    neg_test_data, neg_test_labels = read_tsv(neg_test_file)

    x_train = pos_train_data + neg_train_data
    y_train = pos_train_labels + neg_train_labels

    x_test = pos_test_data + neg_test_data
    y_test = pos_test_labels + neg_test_labels

    print('train data size:{}\ttest data size:{}'.format(len(y_train), len(y_test)))
    print('train data: # of pos:{}\t# of neg:{}\t'.format(y_train.count('pos'), y_train.count('neg')))
    print('test data: # of pos:{}\t# of neg:{}\t'.format(y_test.count('pos'), y_test.count('neg')))
    print('------------------------------------')
    return x_train, y_train, x_test, y_test

# Function to perform sentiment analysis
def do_sa(n, my_classifier, name, my_data):
    x_train, y_train, x_test, y_test = my_data
    print('parameters')
    print('n grams:', n)
    print('classifier:', my_classifier.__class__.__name__)
    print('------------------------------------')

    pipeline = Pipeline([
        ('vect', TfidfVectorizer(min_df=0.0001, max_df=0.5,
                                 analyzer='word', lowercase=False,
                                 ngram_range=(1, n))),
        ('clf', my_classifier),
    ])

    pipeline.fit(x_train, y_train)
    feature_names = pipeline.named_steps['vect'].get_feature_names_out()

    y_predicted = pipeline.predict(x_test)

    # Print the classification report
    print(classification_report(y_test, y_predicted, target_names=['pos', 'neg']))

    # Print the confusion matrix
    cm = confusion_matrix(y_test, y_predicted)
    print(cm)
    print('# of features:', len(feature_names))
    print('sample of features:', random.sample(list(feature_names), 40))
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    return name, n, accuracy, precision, recall

# Main function to orchestrate the entire process
def main(pos_train_file, neg_train_file, pos_test_file, neg_test_file):
    # Load data
    x_train, y_train, x_test, y_test = load(pos_train_file, neg_train_file, pos_test_file, neg_test_file)

    # Preprocess text data
    x_train_processed = [preprocess_arabic_text(text) for text in x_train]
    x_test_processed = [preprocess_arabic_text(text) for text in x_test]

    # Train and evaluate classifier
    my_classifier = MultinomialNB()
    n = 2  # Example n-gram value
    classifier_name = 'Multinomial Naive Bayes'
    data = (x_train_processed, y_train, x_test_processed, y_test)
    results = do_sa(n, my_classifier, classifier_name, data)
    return results





<div style="color:white;display:fill;
            background-color:#3f4d6f;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b> |Setup experiments</b></p>
</div>

In [3]:
ngrams = (1, 2,3)
results = []
pos_training = 'train_pos.tsv'
neg_training = 'train_neg.tsv'

pos_testing = 'test_pos.tsv'
neg_testing = 'test_neg.tsv'

classifiers = [LinearSVC(), SVC(), MultinomialNB(), BernoulliNB(), SGDClassifier(), 
               LogisticRegression(), MLPClassifier(), 
               DecisionTreeClassifier(max_depth=5), RandomForestClassifier(),KNeighborsClassifier(3)]





for g in ngrams:
    dataset = load(pos_training, neg_training, pos_testing, neg_testing)
    for alg in classifiers:
        alg_name = alg.__class__.__name__
        r = do_sa(g, alg, alg_name, dataset)
        results.append(r)
        

train data size:47000	test data size:11751
train data: # of pos:23879	# of neg:23121	
test data: # of pos:5970	# of neg:5781	
------------------------------------
parameters
n grams: 1
classifier: LinearSVC
------------------------------------
              precision    recall  f1-score   support

         pos       0.76      0.78      0.77      5781
         neg       0.78      0.76      0.77      5970

    accuracy                           0.77     11751
   macro avg       0.77      0.77      0.77     11751
weighted avg       0.77      0.77      0.77     11751

[[4524 1257]
 [1451 4519]]
# of features: 13795
sample of features: ['آرمييز', 'ملآيين', 'مجال', 'دربا', 'ٱن', 'الدولة', 'ذم', 'احط', 'أرواح', 'دووم', 'بعدد', 'ارض', 'نرد', 'بال', 'شاف', 'بلادنا', 'المسلسل', 'صرح_الترف_للتبادل', 'خشع', 'للحبيب', 'وأجعلها', 'أصبر', 'تصوير', 'كبد', 'تصعب', 'اياها', 'تكاليف', 'فانصرهم', 'أطفال', 'ألا', 'at', 'طوينا', 'ودا', 'ﻳﺤﺒﻚ', 'لو', 'فيشرب', 'الأعمال', 'للغد', 'تسليم', 'نشوفك']
parameters
n

C:\Users\User\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


              precision    recall  f1-score   support

         pos       0.77      0.78      0.77      5781
         neg       0.78      0.77      0.77      5970

    accuracy                           0.77     11751
   macro avg       0.77      0.77      0.77     11751
weighted avg       0.77      0.77      0.77     11751

[[4482 1299]
 [1375 4595]]
# of features: 13795
sample of features: ['اييه', 'ورجعت', 'بتمنى', 'وأجمل', 'معلش', 'مفيد', 'الويكند', 'ولكم', 'النفسيات', 'وطبعا', 'لأي', 'كذاا', 'ترا', 'ايام', 'خيخه', 'اشد', 'وبأذن', 'الصداع', 'تسمح', 'التجمع', 'عضلة', 'يصف', 'احط', 'ترفع', 'الوضع', 'حلمي', 'أفضل', 'آرمييز', 'عليهما', 'احمد', 'الدرس', 'الشافعي', 'مستواك', 'الكل_رابح_مع_اسرع', 'تف', 'وارهابيين', 'الاسم', 'قرعة', 'وتشويه', 'جاهز']
parameters
n grams: 1
classifier: DecisionTreeClassifier
------------------------------------
              precision    recall  f1-score   support

         pos       0.54      0.96      0.69      5781
         neg       0.85      0.19      0

C:\Users\User\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


              precision    recall  f1-score   support

         pos       0.77      0.77      0.77      5781
         neg       0.78      0.77      0.77      5970

    accuracy                           0.77     11751
   macro avg       0.77      0.77      0.77     11751
weighted avg       0.77      0.77      0.77     11751

[[4480 1301]
 [1370 4600]]
# of features: 27834
sample of features: ['التخرج', 'لنعمة', 'الموجود في', 'تورينا', 'كبير مش', 'كأن', 'داير', 'عوضني خيرا', 'كلماته', 'منا يا', 'وان اليه', 'ضعيفه واضعف', 'ابتعدنا', 'المحترفي', 'ورود', 'اوقات', 'سعودية', 'المدينه', 'ﺿﺮ ﺗﻜﺸﻔﻪ', 'تخونه', 'معقولة', 'فأوجعهم ما', 'مرتاح الحين', 'عطوره', 'وسخونة وهي', 'كيلو', 'مدينه', 'ممتعة', 'تنبييه الساعه', 'وسعيد', 'بطن', 'عيد ميلادي', 'هذا المساء', 'فوقهم', 'صحيحة', 'على الشيطان', 'بسهولة ثقتنا', 'تصلي', 'وصول', 'من شهر']
parameters
n grams: 2
classifier: DecisionTreeClassifier
------------------------------------
              precision    recall  f1-score   support

         pos       

C:\Users\User\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


              precision    recall  f1-score   support

         pos       0.77      0.77      0.77      5781
         neg       0.78      0.77      0.78      5970

    accuracy                           0.77     11751
   macro avg       0.77      0.77      0.77     11751
weighted avg       0.77      0.77      0.77     11751

[[4477 1304]
 [1368 4602]]
# of features: 39232
sample of features: ['القياسي', 'عن حالتي عقب', 'احسن الناس علي', 'وبتروح في', 'الفوز في', 'سأكمل ساعة بلا', 'أول أغلا', 'نفسي لتشكيلة', 'والمالتي', 'أما العبودية فستبقى', 'الفترة الأولى من', 'ورتويت', 'رئيسين في يومين', 'بدل ما اجلس', 'أحد وفوق الجميع', 'العسل كلوو', 'تموت الأخيرة يبدو', 'الدوري السعودي', 'عندهم استعداد بأنهم', 'طول عمرك تذكرين', 'الكرام أنا فارقه', 'مبرووك', 'ورجعت', 'الخوازيق وكان', 'وجهها وغيرت', 'ما لم يحدث', 'إلى أن', 'لايضيق لنا', 'يصعب', 'تكفوون', 'فقط', 'كف', 'لطف الله', 'ضد أحد', 'حالكم كأنه', 'أحد بالله', 'الصمت إما', 'مبرووك لعشاق', 'من السعادة', 'يايمن']
parameters
n grams: 3
classifier: 

<div style="color:white;display:fill;
            background-color:#3f4d6f;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:white;"><b> | Results Summary</b></p>
</div>

In [7]:
print('{0:25}|{1:10}{2:10}{3:10}{4:10}'.format('algorithm', 'ngram', 'accuracy', 'precision', 'recall'))
print('---------------------------------------------------------------------')

# قيمة ngram السابقة
prev_ngram = None

for r in results:
    if prev_ngram is None:
        prev_ngram = r[1]
    elif prev_ngram != r[1]:
        print("---------------------------------------------------------------------") 
        print("---------------------------------------------------------------------") 
    prev_ngram = r[1]
    
    print('{0:25}|{1:10}{2:10.3f}{3:10.3f}{4:10.3f}'.format(r[0], r[1], r[2], r[3], r[4]))


algorithm                |ngram     accuracy  precision recall    
---------------------------------------------------------------------
LinearSVC                |         1     0.770     0.770     0.770
SVC                      |         1     0.800     0.803     0.800
MultinomialNB            |         1     0.761     0.762     0.761
BernoulliNB              |         1     0.758     0.763     0.758
SGDClassifier            |         1     0.764     0.768     0.764
LogisticRegression       |         1     0.766     0.767     0.766
MLPClassifier            |         1     0.772     0.773     0.772
DecisionTreeClassifier   |         1     0.572     0.694     0.572
RandomForestClassifier   |         1     0.798     0.801     0.798
KNeighborsClassifier     |         1     0.698     0.748     0.698
---------------------------------------------------------------------
---------------------------------------------------------------------
LinearSVC                |         2     0.774     0.